## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # TensorFlow (already installed in Colab, but you can upgrade if needed)
# !pip install -q --upgrade tensorflow

# imgaug (compatible with older NumPy)
!pip install -q imgaug==0.4.0

# Downgrade NumPy to avoid issues with imgaug (NumPy ≥ 2.0 breaks imgaug)
!pip install -q numpy==1.26.4


In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

from imgaug import augmenters as iaa ## for augmentation

## Defining paths

In [ ]:
model_name = 'Vgg16_unet_2500'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 10
EPOCHS = 65#100

TOTAL_DATA = 2500
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 1234
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 3
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Augmentations

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Flipud(0.2),  # vertically flip 20% of the images
    iaa.Affine(
        scale={"x": (0.8, 1.1), "y": (0.8, 1.1)},  # zoom in or out (80-120%)
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # width & height shift (-20 to +20%)
        rotate=(-15, 15),  # rotation (-45 to 45 degrees)
        shear=(-16, 16)  # shear (-16 to 16 degrees)
    ),
    iaa.ElasticTransformation(alpha=(0, 2.0), sigma=0.25),  # apply elastic deformations
    iaa.GaussianBlur(sigma=(0, 2.0))  # apply Gaussian blur
])

In [ ]:
# Function to augment a batch of images and masks
def augment_batch(images, masks, seq, batch_size):
    augmented_images = []
    augmented_masks = []
    while len(augmented_images) < batch_size:
        aug_images, aug_masks = seq(images=images, segmentation_maps=np.expand_dims(masks, axis=-1))
        augmented_images.extend(aug_images)
        augmented_masks.extend(aug_masks)

        if len(augmented_images) >= batch_size:
            break

    # Ensure we only take as many as we need to reach the desired batch_size
    augmented_images = augmented_images[:batch_size]
    augmented_masks = augmented_masks[:batch_size]

    return np.array(augmented_images), np.squeeze(np.array(augmented_masks), axis=-1)

In [ ]:
num_augmented_images_needed = TOTAL_DATA  - len(x_train)  # Calculate how many augmented images we need
if num_augmented_images_needed > 0:
    augmented_x, augmented_y = augment_batch(x_train, y_train, seq, num_augmented_images_needed)
    # Concatenate the original and augmented datasets
    combined_x_train = np.concatenate((x_train, augmented_x), axis=0)
    combined_y_train = np.concatenate((y_train, augmented_y), axis=0)
else:
    combined_x_train = x_train
    combined_y_train = y_train

In [ ]:
x_train = combined_x_train
y_train = combined_y_train

## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)


# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 2500 original images into x_train
Loaded 2500 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )

    my_callbacks = [best_model_callback , reduce_LR]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### get_unet_vgg16--works with batch30-augmented data (could not check with 40--memory issue)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x


from tensorflow.keras.applications import  VGG16

# Define the U-Net model with pre-trained VGG16 weights
def get_unet_vgg16(n_filters=16, batchnorm=True):
    # Load pre-trained VGG16 model (excluding top layers)
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=MODEL_INPUT_SIZE)

    # Encoder (Contracting Path)
    encoder = vgg16.get_layer('block4_pool').output

    # Bottleneck
    bottleneck = Conv2D(512, (3, 3), activation='relu', padding='same')(encoder)

    # Decoder (Expansive Path)
    x = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(bottleneck)
    x = Concatenate()([x, vgg16.get_layer('block3_pool').output])
    #x = Conv2D(n_filters * 16, (3, 3), activation='relu', padding='same')(x)
    x = conv2d_block(x, 256, kernel_size = 3, batchnorm = batchnorm)

    x = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(x)
    x = Concatenate()([x, vgg16.get_layer('block2_pool').output])
    #x = Conv2D(n_filters * 8, (3, 3), activation='relu', padding='same')(x)
    x = conv2d_block(x, 128, kernel_size = 3, batchnorm = batchnorm)

    x = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(x)
    x = Concatenate()([x, vgg16.get_layer('block1_pool').output])
    #x = Conv2D(n_filters * 4, (3, 3), activation='relu', padding='same')(x)
    x = conv2d_block(x, 64, kernel_size = 3, batchnorm = batchnorm)

    x = Conv2DTranspose(32, (3, 3), strides = (2, 2), padding = 'same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Output layer
    output = Conv2D(NUM_CLASSES-1, (1, 1), activation='sigmoid')(x)  #sigmoid because 2 classes, softmax for multiple class

    model = Model(inputs=vgg16.input, outputs=output)


    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],#, iou_score_binary, iou_score1],
                  optimizer=optimizer)

    return model

## Train and Results

In [ ]:
# model= get_basic_unet() ## Model1
model= get_unet_vgg16() ## Model2
# model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
# model = get_ResUNet_v3() ## Model5
# model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
# model = get_unet_InceptionResNetV2() ## Model8

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 256, 256,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 128, 128,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 128, 128,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 128, 128,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 64, 64,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 64, 64,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 64, 64,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 64, 64,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 32, 32,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 32, 32,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 32, 32,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 32, 32,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 16, 16,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 16, 16,    │  2,359,808 │ block4_pool[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │  1,179,904 │ conv2d[0][0]    

 Total params: 13,907,745 (53.05 MB)

 Trainable params: 13,905,953 (53.05 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
# training the model and saving the best model as a check point
best_model_name, train_history, total_time = fit_and_save_best_model(model_name, model)

Epoch 1/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9647 - binary_io_u: 0.6286 - f1_score: 0.3590 - loss: 0.1088
Epoch 1: val_binary_io_u improved from -inf to 0.77372, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 143s 463ms/step - accuracy: 0.9647 - binary_io_u: 0.6291 - f1_score: 0.3601 - loss: 0.1086 - val_accuracy: 0.9787 - val_binary_io_u: 0.7737 - val_f1_score: 0.6831 - val_loss: 0.0882 - learning_rate: 0.0010
Epoch 2/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9831 - binary_io_u: 0.7973 - f1_score: 0.7491 - loss: 0.0502
Epoch 2: val_binary_io_u improved from 0.77372 to 0.78549, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 357ms/step - accuracy: 0.9831 - binary_io_u: 0.7973 - f1_score: 0.7491 - loss: 0.0502 - val_accuracy: 0.9829 - val_binary_io_u: 0.7855 - val_f1_score: 0.6921 - val_loss: 0.0520 - learning_rate: 0.0010
Epoch 3/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9840 - binary_io_u: 0.8065 - f1_score: 0.7618 - loss: 0.0472
Epoch 3: val_binary_io_u improved from 0.78549 to 0.79883, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 356ms/step - accuracy: 0.9840 - binary_io_u: 0.8065 - f1_score: 0.7618 - loss: 0.0472 - val_accuracy: 0.9831 - val_binary_io_u: 0.7988 - val_f1_score: 0.7171 - val_loss: 0.0552 - learning_rate: 0.0010
Epoch 4/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 0.9846 - binary_io_u: 0.8115 - f1_score: 0.7688 - loss: 0.0454
Epoch 4: val_binary_io_u did not improve from 0.79883
200/200 ━━━━━━━━━━━━━━━━━━━━ 69s 342ms/step - accuracy: 0.9846 - binary_io_u: 0.8115 - f1_score: 0.7689 - loss: 0.0454 - val_accuracy: 0.9817 - val_binary_io_u: 0.7966 - val_f1_score: 0.7185 - val_loss: 0.0546 - learning_rate: 0.0010
Epoch 5/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 0.9852 - binary_io_u: 0.8183 - f1_score: 0.7779 - loss: 0.0437
Epoch 5: val_binary_io_u improved from 0.79883 to 0.80992, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 354ms/step - accuracy: 0.9852 - binary_io_u: 0.8183 - f1_score: 0.7780 - loss: 0.0437 - val_accuracy: 0.9841 - val_binary_io_u: 0.8099 - val_f1_score: 0.7297 - val_loss: 0.0472 - learning_rate: 0.0010
Epoch 6/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 0.9857 - binary_io_u: 0.8236 - f1_score: 0.7852 - loss: 0.0421
Epoch 6: val_binary_io_u improved from 0.80992 to 0.81347, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


...

200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 354ms/step - accuracy: 0.9912 - binary_io_u: 0.8883 - f1_score: 0.8708 - loss: 0.0232 - val_accuracy: 0.9893 - val_binary_io_u: 0.8642 - val_f1_score: 0.8066 - val_loss: 0.0304 - learning_rate: 7.8125e-06
Epoch 45/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8883 - f1_score: 0.8708 - loss: 0.0231
Epoch 45: val_binary_io_u improved from 0.86423 to 0.86426, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5



Epoch 45: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
200/200 ━━━━━━━━━━━━━━━━━━━━ 72s 358ms/step - accuracy: 0.9912 - binary_io_u: 0.8883 - f1_score: 0.8709 - loss: 0.0231 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8067 - val_loss: 0.0304 - learning_rate: 7.8125e-06
Epoch 46/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8885 - f1_score: 0.8711 - loss: 0.0231
Epoch 46: val_binary_io_u improved from 0.86426 to 0.86431, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 355ms/step - accuracy: 0.9912 - binary_io_u: 0.8885 - f1_score: 0.8711 - loss: 0.0231 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8065 - val_loss: 0.0303 - learning_rate: 3.9063e-06
Epoch 47/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8885 - f1_score: 0.8711 - loss: 0.0231
Epoch 47: val_binary_io_u improved from 0.86431 to 0.86432, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 353ms/step - accuracy: 0.9912 - binary_io_u: 0.8885 - f1_score: 0.8712 - loss: 0.0231 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8065 - val_loss: 0.0304 - learning_rate: 3.9063e-06
Epoch 48/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8886 - f1_score: 0.8712 - loss: 0.0231
Epoch 48: val_binary_io_u did not improve from 0.86432
200/200 ━━━━━━━━━━━━━━━━━━━━ 69s 343ms/step - accuracy: 0.9912 - binary_io_u: 0.8886 - f1_score: 0.8712 - loss: 0.0231 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8065 - val_loss: 0.0304 - learning_rate: 3.9063e-06
Epoch 49/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8886 - f1_score: 0.8713 - loss: 0.0230
Epoch 49: val_binary_io_u improved from 0.86432 to 0.86432, saving model to /content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation_AUC/Vgg16_unet_2500/saved_models/best_Vgg16_unet_2500.h5


Epoch 49: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.
200/200 ━━━━━━━━━━━━━━━━━━━━ 71s 353ms/step - accuracy: 0.9912 - binary_io_u: 0.8886 - f1_score: 0.8713 - loss: 0.0230 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8065 - val_loss: 0.0304 - learning_rate: 3.9063e-06
Epoch 50/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8888 - f1_score: 0.8715 - loss: 0.0230
Epoch 50: val_binary_io_u did not improve from 0.86432
200/200 ━━━━━━━━━━━━━━━━━━━━ 69s 343ms/step - accuracy: 0.9912 - binary_io_u: 0.8888 - f1_score: 0.8715 - loss: 0.0230 - val_accuracy: 0.9893 - val_binary_io_u: 0.8643 - val_f1_score: 0.8063 - val_loss: 0.0303 - learning_rate: 1.9531e-06
Epoch 51/65
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9912 - binary_io_u: 0.8887 - f1_score: 0.8714 - loss: 0.0230
Epoch 51: val_binary_io_u did not improve from 0.86432
200/200 ━━━━━━━━━━━━━━━━━━━━ 68s 342ms/step - accuracy: 0.9912 - binary_io_

## Loading the best model

In [ ]:
##Now, load the best model
best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
                                  custom_objects={'f1_score': f1_score,
                                                  'binary_io_u':iou_score_binary})

# best_model =model

## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step


# AUC

In [ ]:
# Add this section *after* prediction is made with the best model
# i.e., after: predictions = best_model.predict(x_test, verbose=1)

from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9896, Std=0.0003, 95% CI=(0.9892, 0.9901)
